In [ ]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.29:8501

2025-05-13 16:06:58.051 Examining the path of torch.classes raised: Tried to instantiate class '__path__._path', but it does not exist! Ensure that it is registered via torch::class_
2025-05-13 16:07:21.092 Uncaught app exception
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 542, in _run_script
    exec(code, module.__dict__)
  File "/Users/hariririski/S2KecerdasanBuatan/2. Semester 2/MMAI1010 Visi Komputer/4_Final/Upload/app.py", line 151, in <module>
    q0 = eval_q_gray(orig8)
         ^^^^^^^^^^^^^^^^^^
  File "/Users/hariririski/S2KecerdasanBuatan/2. Semester 2/MMAI1010 Visi Komputer/4_Final/Upload/app.py", line 25, in eval_q_gray
    return {"niqe": round(niqe(t).item(), 2), "piqe": round(piqe(t).item(), 2)}
                          ^^^^^^^
  File "/opt/

In [20]:
import pandas as pd

# ─────────── 1. KONFIGURASI ───────────
csv_gray = "out/final_P1.csv"
csv_rgb  = "out_rgb/final_P10.csv"

# ─────────── 2. LOAD DATASET ───────────
df_gray = pd.read_csv(csv_gray)
df_rgb  = pd.read_csv(csv_rgb)

# Tambah skor gabungan: semakin kecil lebih baik
df_gray["avg_score"] = (df_gray["fin_niqe"] + df_gray["fin_piqe"]) / 2
df_rgb["avg_score"] = (df_rgb["fin_niqe"] + df_rgb["fin_piqe"]) / 2 - 0.01 * df_rgb["fin_colorfulness"]

# ─────────── 3. FUNGSI PILIH TERBAIK ───────────
def best_per_class(df, prefix_list):
    result = []
    for prefix in prefix_list:
        sub = df[df["filename"].str.startswith(prefix)]
        best = sub.nsmallest(1, "avg_score")
        result.append(best)
    return pd.concat(result)

# ─────────── 4. PILIH TERBAIK PER KELAS ───────────
classes = ["no_", "minor_", "major_"]
best_gray = best_per_class(df_gray, classes).copy()
best_rgb  = best_per_class(df_rgb,  classes).copy()

# ─────────── 5. CETAK HASIL ───────────
def show_table(df, label):
    print(f"\n📊 {label} - Pipeline {df['pipeline'].iloc[0]}")
    for _, row in df.iterrows():
        print(f"{row['filename']:15} | NIQE: {row['orig_niqe']:.2f} → {row['fin_niqe']:.2f} | "
              f"PIQE: {row['orig_piqe']:.2f} → {row['fin_piqe']:.2f} | "
              f"{'CF: '+str(round(row['orig_colorfulness'],2))+' → '+str(round(row['fin_colorfulness'],2)) if 'fin_colorfulness' in row else ''} "
              f"| Avg Score: {row['avg_score']:.2f}")

show_table(best_gray, "Grayscale")
show_table(best_rgb, "RGB")


📊 Grayscale - Pipeline P1
no_082.tiff     | NIQE: 9.99 → 7.64 | PIQE: 94.98 → 14.47 |  | Avg Score: 11.05
minor_075.tiff  | NIQE: 9.88 → 7.98 | PIQE: 58.74 → 15.72 |  | Avg Score: 11.85
major_069.tiff  | NIQE: 9.99 → 7.54 | PIQE: 65.65 → 15.29 |  | Avg Score: 11.41

📊 RGB - Pipeline P10
no_064.tiff     | NIQE: 11.98 → 6.69 | PIQE: 81.63 → 21.21 | CF: 87.78 → 82.92 | Avg Score: 13.12
minor_001.tiff  | NIQE: 11.32 → 6.39 | PIQE: 77.28 → 19.96 | CF: 99.17 → 99.85 | Avg Score: 12.17
major_060.tiff  | NIQE: 11.32 → 6.39 | PIQE: 77.28 → 19.96 | CF: 99.17 → 99.85 | Avg Score: 12.17
